## Auditing and cleaning data before EDA in Tableau

The goal of this process is to ensure the data is as clean as possible to create visualizations in Tableau.

In [87]:
import os
import pandas as pd
import numpy as np

os.listdir()

['.git',
 '.ipynb_checkpoints',
 '2018-2010_export.csv',
 '2018-2010_import.csv',
 'audit_clean.ipynb',
 'create.sql']

In [2]:
exports = pd.read_csv('2018-2010_export.csv')
imports = pd.read_csv('2018-2010_import.csv')

print("EXPORTS:")
display(exports.head())
print("IMPORTS:")
display(imports.head())

EXPORTS:


,HSCode,Commodity,value,country,year
0,2,MEAT AND EDIBLE MEAT OFFAL.,0.18,AFGHANISTAN TIS,2018
1,3,"FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT...",0.00,AFGHANISTAN TIS,2018
2,4,DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY; EDI...,12.48,AFGHANISTAN TIS,2018
3,6,LIVE TREES AND OTHER PLANTS; BULBS; ROOTS AND ...,0.00,AFGHANISTAN TIS,2018
4,7,EDIBLE VEGETABLES AND CERTAIN ROOTS AND TUBERS.,1.89,AFGHANISTAN TIS,2018


IMPORTS:


,HSCode,Commodity,value,country,year
0,5,"PRODUCTS OF ANIMAL ORIGIN, NOT ELSEWHERE SPECI...",0.00,AFGHANISTAN TIS,2018
1,7,EDIBLE VEGETABLES AND CERTAIN ROOTS AND TUBERS.,12.38,AFGHANISTAN TIS,2018
2,8,EDIBLE FRUIT AND NUTS; PEEL OR CITRUS FRUIT OR...,268.60,AFGHANISTAN TIS,2018
3,9,"COFFEE, TEA, MATE AND SPICES.",35.48,AFGHANISTAN TIS,2018
4,11,PRODUCTS OF THE MILLING INDUSTRY; MALT; STARCH...,NaN,AFGHANISTAN TIS,2018


## Handling missing data

In assessing the null values we will be addressing each of the following points:

1. Which features contain null values?
2. How many rows contain null values? (What percentage?)
3. Why are there null values? (Does it make sense?)
4. Final Decision (Drop, Imputation Strategy)

**1. Which features contain null values?**

In [3]:
# Columns with null in exports
print(exports.isnull().any(),end='\n')
# Columns with null in imports
print(imports.isnull().any())

HSCode       False
Commodity    False
value         True
country      False
year         False
dtype: bool
HSCode       False
Commodity    False
value         True
country      False
year         False
dtype: bool


The only columns with Null values for both datasets is the variable `value`

**2. How many rows contain null values? (What percentage?)**

To aid in our decision on how to handle the null values we will examine the proportion of rows with missing values.

In [4]:
n_exports = exports.shape[0]
null_exports = exports[exports.value.isnull()].shape[0]
null_percent_exports = round(null_exports / n_exports, 2) * 100
n_imports = imports.shape[0]
null_imports = imports[imports.value.isnull()].shape[0]
null_percent_imports = round(null_imports / n_imports, 2) * 100

print("PERCENT MISSING IN EXPORTS:", null_percent_exports,"%", str(null_exports)+"/"+str(n_exports))
print("PERCENT MISSING IN IMPORTS:", null_percent_imports,"%", str(null_imports)+"/"+str(n_imports))

PERCENT MISSING IN EXPORTS: 10.0 % 14037/137023
PERCENT MISSING IN IMPORTS: 15.0 % 14027/93095


Both datasets contain over 14K missing values. This constitutes 10-15% of the data. This is a very significant amount of missing values and should be investigated further.

In [54]:
exports['country'].head().value_counts()['AFGHANISTAN TIS']

5

**3. Why are there null values? (Does it make sense?)**

In [99]:
missing_edf = exports[exports.value.isnull()]
missing_idf = imports[imports.value.isnull()]

n_missing_edf = [x for x in missing_edf.country.value_counts().items()]
n_missing_idf = [x for x in missing_idf.country.value_counts().items()]

Ps = []
for v in n_missing_edf:
    country = v[0]
    value = v[1]
    total = exports['country'].value_counts()[country]
    value_string = str(value) + "/" + str(total)
    p = round((value / total), 2)
    Ps.append((country, p, value_string))
nan_df=pd.DataFrame(Ps, columns=['country','NaN_ratio','totals']).sort_values('NaN_ratio',ascending=False)

In [100]:
nan_df[nan_df.NaN_ratio > 0.4]

,country,NaN_ratio,totals
242,GUERNSEY,0.75,3/4
192,SAHARWI A.DM RP,0.62,26/42
237,CHANNEL IS,0.60,6/10
209,NEUTRAL ZONE,0.57,16/28
236,PACIFIC IS,0.55,6/11
7,MONTSERRAT,0.55,116/210
235,FR S ANT TR,0.55,6/11
196,TUVALU,0.53,24/45
232,FALKLAND IS,0.53,8/15
51,ANGUILLA,0.53,83/156


**4. Final Decision (Drop, Imputation Strategy)**

Looking at the countries with more than 40% null values it would appear many of the values are near or at zero. We will assume null trade values to be zero and treat zeros as not a transaction and remove them from the data set.

In [90]:
imports[(imports.value.isna())].shape

(14027, 5)

In [96]:
imports[(imports.value == 0)].shape

(10454, 5)

In [98]:
imports[(imports.value == 0) | (imports.value.isna())].shape[0] / imports.shape[0]

0.2629679359793759

In [101]:
exports[(exports.value == 0) | (exports.value.isna())].shape[0] / exports.shape[0]

0.2181969450384242

In [108]:
def drop_rows(data):
    data = data.dropna()
    data = data[data.value != 0]
    
    return data

imports=drop_rows(imports)
exports=drop_rows(exports)

### Export cleaned dataset

In [109]:
imports.to_csv('2018-2010_import.csv',index=False)
exports.to_csv('2018-2010_export.csv',index=False)

PermissionError: [Errno 13] Permission denied: '2018-2010_export.csv'